(COMPPARALELOCPUSSMC)=

# 5.4 Cómputo en paralelo usando CPUs en un sistema de memoria compartida (SMC)

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

Nota generada a partir de [liga1](https://www.dropbox.com/s/oauifmx3e19ofyq/2.3.Sistemas_de_memoria_compartida_Pthreads.pdf?dl=0), [liga2](https://www.dropbox.com/s/vcxbrqkk6x946d7/2.4.Sistemas_de_memoria_compartida_openMP.pdf?dl=0), [liga3](https://www.dropbox.com/s/v4ub0p3ndf7w1p0/2.2.Sistemas_de_memoria_distribuida_MPI.pdf?dl=0)

```{admonition} Al final de esta nota el y la lectora:
:class: tip

*

```

Se presentan códigos y sus ejecuciones en una máquina `m4.16xlarge` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI `opt2-aws-educate-openblas-04-04-2021` de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_profiling_and_BLAS.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_profiling_and_BLAS.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-educate-openblas-04-04-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=OpenBLAS
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `m4.16xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          64
On-line CPU(s) list:             0-63
Thread(s) per core:              2
Core(s) per socket:              16
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2290.913
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.03
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       1 MiB
L1i cache:                       1 MiB
L2 cache:                        8

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.11.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 256GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 16GiB
          width: 64 bits
     *-bank:4
          description: DIMM RAM
          physical id: 4
          slot: DIMM

In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-140 5.4.0-1038-aws #40-Ubuntu SMP Fri Feb 5 23:50:40 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con `import`. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## Sistemas de memoria compartida (SMC)

Un SMC en general se ve como el siguiente dibujo:

<img src="https://dl.dropboxusercontent.com/s/ao3if8tzwsvzfi7/shared_memory_sistems.png?dl=0" heigth="500" width="500">

El dibujo anterior es un SMC con acceso uniforme a la memoria ([UMA](https://en.wikipedia.org/wiki/Uniform_memory_access)), esto es, cada proceso o *thread* creado en el procesador o *core* accede con las mismas velocidades a la memoria.

La **comunicación** en este tipo de sistemas depende si se utilizan procesos o *threads* ya que aunque los procesos generados por un proceso principal tienen acceso a la memoria, los cambios/actualizaciones que haga uno de ellos a una variable no lo llegan a ver los otros procesos. Esto es distinto con los *threads* dado que un cambio que realice un *thread* en una variable sí lo pueden ver los otros *threads*. Lo anterior se debe a las distintas direcciones de memoria que utilizan los procesos vs la misma dirección de memoria que utilizan los *threads*.

```{admonition} Definición

Las variables que pueden ser accesadas por todos los *threads* en un SMC se les nombra **variables compartidas**.

```

```{admonition} Observación
:class: tip

Hay paqueterías que crean subprocesos a partir de un proceso principal en lugar de *threads*. En este caso en lugar de un *fork* se realiza un *spawn*. Los subprocesos pueden ver cambios a variables hechos por otros. 

Ver [Contexts and start methods](https://docs.python.org/3.9/library/multiprocessing.html#contexts-and-start-methods) y [stackoverflow: difference between threadpool vs pool in python multiprocessing](https://stackoverflow.com/questions/46045956/whats-the-difference-between-threadpool-vs-pool-in-python-multiprocessing-modul) para diferencias entre *fork* y *spawn*.

```

Trabajar sobre SMC tiene ventajas y desventajas. Una ventaja es facilidad de comunicación y una desventaja es la **coordinación** para ejecutar instrucciones. Por ejemplo, con variables compartidas se puede realizar la comunicación entre los *threads*, sin embargo, para el uso de tales variables por diferentes *threads* debemos crear candados, *locks*. Lo anterior surge pues si un *thread* con etiqueta $1$ accede a una variable compartida, otro *thread* con etiqueta $2$, no podrá utilizarla hasta que el thread $1$ finalice de usarla. Ver [Thread Safety](https://en.wikipedia.org/wiki/Thread_safety) y [Race Condition](https://en.wikipedia.org/wiki/Race_condition#Computing).

```{admonition} Definición

En una *race condition* múltiples *threads* intentan acceder a un recurso compartido, al menos uno de los accesos resulta en una modificación al recurso y posteriormente los siguientes accesos pueden no ver la modificación. A la sección del código que causa la *race condition* se le nombra *critical section*. Las *critical sections* se ejecutan con código secuencial o con *locks* para evitar las *race conditions*.

```

```{admonition} Comentarios

* Para crear procesos o *threads* en los lenguajes de programación de *C, Python* o *R*, utilizamos las librerías, API's o extensiones vía paquetes a tales lenguajes. Lo anterior se debe a que *C, Python* y *R* en sus implementaciones más utilizadas o estándar, fueron diseñados con el propósito de utilizarse sobre sistemas con un sólo procesador. En distintas implementaciones de los lenguajes hay soporte para SMC. 

* Ejemplos de máquinas con SMC son las laptops, los celulares, máquinas de escritorio con más de un *core*.

* Otro tipo de SMC se puede representar con el siguiente dibujo:

<img src="https://dl.dropboxusercontent.com/s/dqwdqdxiecj91vy/shared_memory_systems_2.png?dl=0" heigth="500" width="500">

en el que los procesos o *threads* pueden acceder a la memoria en una forma no uniforme [(NUMA)](https://en.wikipedia.org/wiki/Non-uniform_memory_access). Una de las diferencias que se tienen entre un NUMA y un UMA es la tasa de transferencia de datos para *cores* que están más cercanos a una memoria.

* Un sistema de memoria distribuida (SMD) tiene una conexión, por ejemplo vía una *network*, entre pares de *core-memoria* y en general se ve como el siguiente dibujo:

<img src="https://dl.dropboxusercontent.com/s/iky7af1m3dcj3e0/distributed_memory_systems.png?dl=0" heigth="600" width="600">

La memoria en un SMD asociada al *core* sólo puede ser accesada por éste y es inaccesible a los demás *cores*, esto es, se tiene una memoria "privada". Lo anterior contrasta con los SMC en los que todos los *cores* accesan a una memoria compartida. En un SMD el cómputo distribuido crea múltiples procesos a partir de un proceso principal. En un SMC el cómputo paralelo crea múltiples procesos o *threads*. Un ejemplo de un SMD es un clúster de máquinas. Cada máquina en el clúster puede ser un SMC y por tanto los procesos en cada máquina tienen la capacidad de crear procesos o *threads*. En este caso se tiene un sistema híbrido SMD y SMC. Un programa diseñado para ejecutarse en un SMD puede ejecutarse en un SMC pues se divide su memoria de forma lógica en espacios de memoria privados para los *threads*.

```

## [OpenMP](http://www.openmp.org/)

Es una extensión al lenguaje *C* y es una API para cómputo en paralelo en un sistema de memoria compartida, *aka, shared memory parallel programming* con CPUs. Lo anterior *OpenMP* lo posibilita con el *threading*: *fork* y *join* de *threads* a partir de un proceso principal.

<img src="https://dl.dropboxusercontent.com/s/0vnjfdk7fo62m8h/threading.png?dl=0" heigth="400" width="400">

Ver {ref}`Threading o Hyperthreading <THREADINGHYPER>`.

* Las siglas *MP* se refieren a *multiprocessing*, un sinónimo de *shared memory parallel computing*. *OpenMP* se utiliza en un SMC por lo que cada *thread* tiene acceso a la memoria.

* Algunas características de *OpenMP* son:

    * Paralelización de ciclos *for* secuenciales en los que las iteraciones son independientes una de la otra de forma simple.

    * Paralelización de tareas y sincronización explícita de *threads*.

```{margin}

Las directivas son instrucciones que indican al [preprocesador](https://en.wikipedia.org/wiki/Preprocessor) (vía la compilación) que ejecutaremos una instrucción que no se encuentra en la especificación básica del lenguaje C. Ver [C preprocessor](https://en.wikipedia.org/wiki/C_preprocessor).

```

* Provee **directivas** vía `#pragma omp` para el cómputo en paralelo en un SMC.

```{admonition} Comentario

Los `pragma` se utilizan para extender la funcionalidad de *C* pues no son parte de su especificación básica. Las versiones más recientes del compilador `gcc` soportan a los `pragma` y todas las *preprocessor directives* son por *default* de longitud una línea.

Ver [pragmas](https://gcc.gnu.org/onlinedocs/cpp/Pragmas.html).

```

### Directiva parallel

### Ejemplo: *Hello world*

`hello_world_omp.c`

In [1]:
%%file hello_world_omp.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h> 

void Hello(void);
int main(){

    #pragma omp parallel
        Hello();
    
    return 0;
}

void Hello(void){
    int my_rank = omp_get_thread_num(); 
    int num_th = omp_get_num_threads(); 
    printf("Hola del thread: %d de %d\n", my_rank, num_th);
}


Writing hello_world_omp.c


In [2]:
%%bash
gcc -Wall -fopenmp hello_world_omp.c -o hello_world_omp.out

In [3]:
%%bash
./hello_world_omp.out

Hola del thread: 5 de 64
Hola del thread: 59 de 64
Hola del thread: 24 de 64
Hola del thread: 47 de 64
Hola del thread: 32 de 64
Hola del thread: 27 de 64
Hola del thread: 22 de 64
Hola del thread: 60 de 64
Hola del thread: 40 de 64
Hola del thread: 45 de 64
Hola del thread: 51 de 64
Hola del thread: 58 de 64
Hola del thread: 53 de 64
Hola del thread: 44 de 64
Hola del thread: 10 de 64
Hola del thread: 46 de 64
Hola del thread: 11 de 64
Hola del thread: 4 de 64
Hola del thread: 15 de 64
Hola del thread: 31 de 64
Hola del thread: 8 de 64
Hola del thread: 13 de 64
Hola del thread: 16 de 64
Hola del thread: 7 de 64
Hola del thread: 21 de 64
Hola del thread: 1 de 64
Hola del thread: 14 de 64
Hola del thread: 3 de 64
Hola del thread: 37 de 64
Hola del thread: 43 de 64
Hola del thread: 18 de 64
Hola del thread: 17 de 64
Hola del thread: 62 de 64
Hola del thread: 25 de 64
Hola del thread: 29 de 64
Hola del thread: 0 de 64
Hola del thread: 30 de 64
Hola del thread: 54 de 64
Hola del thread: 2 

````{admonition} Comentarios

* `omp.h` es un *header file* con prototipos y definiciones de macros para uso de la librería de funciones y macros de *OpenMP*.

* La función `Hello` será ejecutada por los *threads*.

* La función `omp_get_thread_num` da el *rank* asignado por el *run time system* a cada *thread*.

* Con la función `omp_get_num_threads` se obtiene el número de *threads* que realizaron un *fork* del *thread* principal.

* Obsérvese que para la compilación se utilizó la *flag* `-fopenmp` para soporte de *OpenMP*.

* Dependiendo del número de cores de nuestro sistema tendremos diferentes número de `printf`'s.

* Lo que continúa a la línea de `#pragma omp parallel` es un *structured block*, esto es, un *statement* o conjunto de *statements* que tienen un punto de entrada y un punto de salida. En el caso anterior sólo se llama a la función `Hello`, no se permiten statements como el siguiente:

```C
#pragma omp parallel

if(...) break;

```

ni tampoco:
    
```C
#pragma omp parallel

    {
        if(variable == valor) return 1;
        return -1;
    }

```

````

A continuación de la directiva `parallel` podemos usar diferentes tipos de *clauses*. Una *clause* en *OpenMP* es un texto que modifica una directiva. Por ejemplo, podemos usar la *clause* `num_threads` para especificar el número de threads que ejecutarán el *structured block*.

### *Clause* `num_threads`

### Ejemplo: *Hello world*

`hello_world_omp_num_threads.c`

In [4]:
%%file hello_world_omp_num_threads.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h> 

void Hello(void); 
int main(){
    int n_threads = 5;
    #pragma omp parallel num_threads(n_threads) 
        Hello();
    
    return 0;
}

void Hello(void){
    int my_rank = omp_get_thread_num(); 
    int num_th = omp_get_num_threads(); 
    printf("Hola del thread: %d de %d\n", my_rank, num_th);
}


Writing hello_world_omp_num_threads.c


In [5]:
%%bash
gcc -Wall -fopenmp hello_world_omp_num_threads.c -o hello_world_omp_num_threads.out

In [6]:
%%bash
./hello_world_omp_num_threads.out

Hola del thread: 0 de 5
Hola del thread: 1 de 5
Hola del thread: 2 de 5
Hola del thread: 3 de 5
Hola del thread: 4 de 5


### *Reduction clause* 

*OpenMP* provee la *reduction clause* para aplicar la operación de suma (operador binario) a cada resultado calculado de un *thread* de forma repetida y almacenar en una variable la respuesta.

Nombramos *reduction variable* a la variable que almacenará los resultados intermedios calculados por cada *thread* y *reduction operator* a la operación binaria (por ejemplo una suma o multiplicación) que se aplica repetidamente a una secuencia de operandos para obtener un resultado, en un proceso que se le nombra *reduction*. 

Por ejemplo, si `A` es un arreglo de `n` enteros, el cálculo:

```C
int sum = 0;
for(i=0;i<n;i++)
    sum += A[i];
``` 

es un proceso de *reduction* en el que el *reduction operator* es la suma.

En openMP utilizamos la *reduction clause* en la *parallel directive*.


```C
sum_shared = 0.0
#pragma omp parallel num_threads(conteo_threads) reduction(+: sum_shared)
    sum_shared += Rcf_parallel(a,h_hat,n_subintervals_per_core);

```

Con la *reduction clause* openMP crea una variable privada por cada *thread* y el *run time system* almacena el resultado de cada *thread* en esta variable. *openMP* también crea una *critical section* y los valores almacenados en las variables privadas son sumadas en esta *critical section* y almacenados en la *reduction variable* `sum_shared`. El *reduction operator* es: `+`.

La sintaxis de la *reduction clause* es:

```C
reduction( <operator>: <variable list>)
```

El *reduction operator* puede ser cualquiera de los operadores: `+,*,-,&,|,^,&&,||`. Cabe señalar que el proceso de *reduction* asume que los operadores utilizados cumplen con la propiedad asociativa (por ejemplo, el operador de resta no cumple con esto).


La variable que está en la *reduction clause* es compartida. Sin embargo una variable privada es creada por cada *thread* en el *team* (**con el mismo nombre** que aparece en la *reduction clause*) y si un *thread* ejecuta un statement en el *parallel block* que involucra a la variable, entonces se utiliza la variable privada y al finalizar el *parallel block*, los valores calculados en las variables privadas son combinados en la variable compartida.

### Ejemplo de uso de nombres de variables al definir variables privadas y compartidas en *reduction clause*

`private_shared_variable_reduction_clause_example.c`

In [7]:
%%file private_shared_variable_reduction_clause_example.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h>

int main(int argc, char *argv[]){
    long n_threads;
    int private_variable;
    int sum_shared = 7;
        
    n_threads = strtol(argv[1], NULL, 10);
    
    printf("variable sum_shared al inicio : %d\n", sum_shared);
    
    #pragma omp parallel num_threads(n_threads) reduction(+: sum_shared)
    {
        int my_rank = omp_get_thread_num();
        if(my_rank==0)
            printf("sum_shared, printf 1 del thread 0: %d\n", sum_shared);
        else
            printf("sum_shared, prinftf 1 thread 1: %d\n", sum_shared);
        sum_shared = (my_rank==0)?10:20;
        if(my_rank==0)
            printf("sum_shared, printf 2 del thread 0: %d\n", sum_shared);
        else
            printf("sum_shared, printf 2 del thread 1: %d\n", sum_shared);
        private_variable = (my_rank==0)?1:2;
        sum_shared += private_variable;
        if(my_rank==0)
            printf("sum_shared, printf 3 del thread 0: %d\n", sum_shared);
        else
            printf("sum_shared, printf 3 del thread 1: %d\n", sum_shared);
    
    }
    printf("sum_shared al final de la directive parallel %d\n", sum_shared);
    
    return 0;
}

Writing private_shared_variable_reduction_clause_example.c


Compilamos:

In [10]:
%%bash
gcc -Wall -fopenmp private_shared_variable_reduction_clause_example.c -o private_shared_variable_reduction_clause_example.out 

Ejecutamos:

In [11]:
%%bash
./private_shared_variable_reduction_clause_example.out 2

variable sum_shared al inicio : 7
sum_shared, printf 1 del thread 0: 0
sum_shared, printf 2 del thread 0: 10
sum_shared, printf 3 del thread 0: 11
sum_shared, prinftf 1 thread 1: 0
sum_shared, printf 2 del thread 1: 20
sum_shared, printf 3 del thread 1: 22
sum_shared al final de la directive parallel 40


```{admonition} Comentario

La variable `sum_shared` se inicializa en 0. En general, las variables privadas creadas para una *reduction clause* son inicializadas al *identity value* para el *operator*. Por ejemplo, si el *operator* es la multiplicación, entonces las variables privadas son inicializadas en 1.

```

### Ejemplo regla compuesta del rectángulo

Para la medición de tiempos se utilizaron las ligas: [measuring-time-in-millisecond-precision](https://stackoverflow.com/questions/16764276/measuring-time-in-millisecond-precision) y [find-execution-time-c-program](https://www.techiedelight.com/find-execution-time-c-program/)

`Rcf_openmp.c`

In [21]:
%%file Rcf_openmp.c
#include<stdio.h>
#include<stdlib.h>
#include<omp.h>
#include<math.h> 
#include<time.h>
#include <sys/time.h>

double Rcf_parallel(double a, double h_hat, int ns_p);

double f(double node);

int main(){
    double sum_shared = 0.0; //shared variable for threads
    double a = 0.0, b = 1.0;
    int n = 1e7; //number of subintervals
    double h_hat = (b-a)/n;
    int n_subintervals_per_core; //number of subintervals assigned to each core
    int n_threads[7] = {0};//n_threads must divide exactly n variable
    int len_n_threads_array = 0;
    double obj = 0.7468241328124271;
    struct timeval start;
    struct timeval end;
    long seconds;
    long long mili;
    int i;
    n_threads[0] = 1;
    n_threads[1] = 2;
    n_threads[2] = 4;
    n_threads[3] = 8;
    n_threads[4] = 16;
    n_threads[5] = 32;
    n_threads[6] = 64;
    len_n_threads_array = sizeof(n_threads)/sizeof(n_threads[0]);
    for(i=0;i<len_n_threads_array;i++){
        n_subintervals_per_core = n/n_threads[i];
        gettimeofday(&start, NULL);
        #pragma omp parallel num_threads(n_threads[i]) reduction(+: sum_shared)
            sum_shared += Rcf_parallel(a,h_hat,n_subintervals_per_core);
        sum_shared = h_hat*sum_shared;
        gettimeofday(&end, NULL);
        seconds = (end.tv_sec - start.tv_sec);
        mili = 1000*(seconds) + (end.tv_usec - start.tv_usec)/1000;
        printf("Integral de %f a %f = %1.15e\n", a,b,sum_shared);
        printf("Error relativo de la solución: %1.15e\n", fabs(sum_shared-obj)/fabs(obj));
        printf("Tiempo de ejecución con %d threads: %lld miliseconds\n", n_threads[i],mili);
        printf("----------------------\n");
        sum_shared = 0.0;
    }
    return 0;
}

double Rcf_parallel(double a, double h_hat, int n_s_c){
    int begin, end;
    int my_rank = omp_get_thread_num();
    double local_int=0;
    int i;
    double x;
    begin = my_rank*n_s_c;
    end = begin + n_s_c; 
    for(i=begin;i<=end-1;i++){
        x = a+(i+1/2.0)*h_hat;
        local_int += f(x);
    }   
    return local_int;
}
        
double f(double node){
    double f_value;
    f_value = exp(-pow(node,2));
    return f_value;
}

Writing Rcf_openmp.c


In [22]:
%%bash
gcc -Wall -fopenmp Rcf_openmp.c -o Rcf_openmp.out -lm

In [23]:
%%bash
./Rcf_openmp.out

Integral de 0.000000 a 1.000000 = 7.468241328123898e-01
Error relativo de la solución: 4.994950214975770e-14
Tiempo de ejecución con 1 threads: 491 miliseconds
----------------------
Integral de 0.000000 a 1.000000 = 7.468241328124163e-01
Error relativo de la solución: 1.441994556109076e-14
Tiempo de ejecución con 2 threads: 239 miliseconds
----------------------
Integral de 0.000000 a 1.000000 = 7.468241328124452e-01
Error relativo de la solución: 2.423145491193603e-14
Tiempo de ejecución con 4 threads: 121 miliseconds
----------------------
Integral de 0.000000 a 1.000000 = 7.468241328124374e-01
Error relativo de la solución: 1.382530863073651e-14
Tiempo de ejecución con 8 threads: 60 miliseconds
----------------------
Integral de 0.000000 a 1.000000 = 7.468241328124298e-01
Error relativo de la solución: 3.567821582125550e-15
Tiempo de ejecución con 16 threads: 30 miliseconds
----------------------
Integral de 0.000000 a 1.000000 = 7.468241328124294e-01
Error relativo de la solución:

```{admonition} Comentarios

* Dividimos el número de subintervalos  **n** entre el número de *threads* que deseamos lanzar, por esto, **n** debe ser **divisible** entre las entradas del *array* `n_threads`. Esta cantidad es el número de subintervalos contiguos que le corresponden a cada *thread*.

* Además, se debe de agregar el resultado de la suma de cada *thread*. Esto es posible realizar de forma sencilla definiendo una variable que sea compartida. Al definir tal variable en la función *main* y antes de un *parallel block* el *default* es que sea considerada como compartida.

* Las variables que son privadas se definen en la función `Rcf_parallel`.

```

```{admonition} Ejercicio
:class: tip

Implementar la regla de Simpson con *OpenMP* utilizando una *reduction clause* en una máquina de AWS con las mismas características que la que se presenta en esta nota y medir tiempo de ejecución.

```

## [Multiprocessing](https://docs.python.org/3.1/library/multiprocessing.html)

La implementación estándar de Python, [CPython](https://github.com/python/cpython) no utiliza múltiples *cores* por *default* para procesamiento. En el apartado de [Cpython-Design](https://en.wikipedia.org/wiki/CPython) se menciona el por qué CPython no soporta ejecución *multithreaded* o *multiprocesses* y a continuación se coloca unos párrafos de tal discusión.

*A particular feature of CPython is that it makes use of a global interpreter lock (GIL) on each CPython interpreter process, which means that within a single process, only one thread may be processing Python bytecode at any one time. This does not mean that there is no point in multithreading; the most common multithreading scenario is where threads are mostly waiting on external processes to complete.*

*For example, imagine when three threads are servicing separate clients. One thread may be waiting for a client to reply, and another may be waiting for a database query to execute, while the third thread is actually processing Python code.*

*However, the GIL does mean that CPython is not suitable for processes that implement CPU-intensive algorithms in Python code that could potentially be distributed across multiple cores. ...*

El módulo *multiprocessing* permite realizar procesamientos basados en procesos o *threads* para compartir trabajo y datos y lidiar con lo mencionado anteriormente. 

Se recomienda usar este módulo para el *shared memory programming* y para trabajos que son demandantes de CPU. Para paralelizar trabajos demandantes en I/O no se recomienda su uso.

```{admonition} Comentario

Otro módulo en *Python* para procesamiento utilizando los cores de tu máquina es [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html) que provee el comportamiento principal de *multiprocessing*. Ver [concurrent-futures-processpoolexecutor-vs-multiprocessing-pool-pool](https://stackoverflow.com/questions/38311431/concurrent-futures-processpoolexecutor-vs-multiprocessing-pool-pool?noredirect=1&lq=1) y [concurrent-futures-vs-multiprocessing-in-python-3](https://stackoverflow.com/questions/20776189/concurrent-futures-vs-multiprocessing-in-python-3) para más sobre *concurrent.futures* y *concurrent.futures* vs *multiprocessing*.

```

## Nota sobre el GIL y *multiprocessing*

Aunque en *Python* los *threads* son nativos del sistema operativo (esto es, no se simulan, son realmente *threads* del sistema operativo creados en el hardware), están limitados por el *global interpreter lock* (GIL), de modo que un sólo *thread* interactúe con un objeto Python en un único tiempo. Esto degrada el *performance* de los programas pues los threads **compiten** por el *GIL* presente en el intérprete de Python, ver [Understanding the Python GIL](http://www.dabeaz.com/GIL/).

Al usar el módulo *multiprocessing* ejecutamos en paralelo un número de intérpretes Python (CPython), cada uno con su propio espacio de memoria privada y su propio GIL que se ejecutan en un instante (y con un *thread*).

```{admonition} Observación
:class: tip

En *multiprocessing* se utilizan subprocesos en lugar de *threads* y en lugar de *fork* se realiza *spawn*.

```

## [Dask](https://docs.dask.org/en/latest/)

## [Parallel](https://www.rdocumentation.org/packages/parallel/versions/3.6.2)

## Referencias de interés

* [Use cases with dask](https://stories.dask.org/en/latest/)

* [dask-tutorial](https://github.com/dask/dask-tutorial)

En *dask* se hace referencia al uso de funciones *pure*. Ver: [Pure Functions by Default](https://distributed.dask.org/en/latest/client.html#pure-functions-by-default) y [Function Purity](https://toolz.readthedocs.io/en/latest/purity.html) para ejemplos de funciones *pure*.

En [Dask JupyterLab Extension](https://www.youtube.com/watch?v=EX_voquHdk0) se muestra cómo instalar la extensión en jupyterlab para dask.

* [snow Simplified](http://www.sfu.ca/~sblay/R/snow.html)

* [Using foreach and iterators for manual parallel execution](https://docs.microsoft.com/en-us/machine-learning-server/r/how-to-revoscaler-distributed-computing-foreach)

Paquete de *R*: [future](https://www.rdocumentation.org/packages/future/versions/1.21.0)

```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Preguntas de comprehensión:**

1)Menciona diferencias que surgen en un programa que se ejecuta en un sistema de memoria compartida contra los que se ejecutan en un sistema de memoria distribuida.

2)¿A qué se le llama no determinismo y da un ejemplo en el que esto surge en un sistema de memoria compartida?

3)¿Qué es una *critical section*? ¿qué es una *race condition*? ¿cómo se puede lidiar con las *critical sections*?

4)¿Cuál es la terminología para nombrar a las variables que pueden ser accesadas por todos los *threads* y para las variables que sólo pueden ser accesadas por un *thread*?

**Referencias:**

1. P. Pacheco, An Introduction to Parallel Programming, Morgan Kaufmann, 2011.

2. M. Gorelick, I. Ozsvald, High Performance Python, O'Reilly Media, 2014.

3. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

4. B. W. Kernighan, D. M. Ritchie, The C Programming Language, Prentice Hall Software Series, 1988

5. [C/extensiones_a_C/openMP](https://github.com/palmoreck/programming-languages/tree/master/C/extensiones_a_C/openMP)
